KUHN POKER
1. A deck of 3 cards is used.
2. Each player is dealt one card.
3. Player 1 can either bet (1) or check (0).
  If player 1 checks, player 2 can either bet or check.
  If player 2 checks, the player with the higher card wins the pot.
  If player 2 bets, the action goes back to player 1 who can either call or fold.
4. If player 1 bets, player 2 can call or fold.

In [3]:
import numpy as np
from random import shuffle

NUM_ACTIONS, NUM_CARDS = 2, 3
action_map = {0: 'p', 1: 'b'}

In [4]:
class Node:
  def __init__(self):
    self.regret_sum = np.zeros(NUM_ACTIONS)
    self.strategy_sum = np.zeros(NUM_ACTIONS)
    self.visits = 0

  def get_strategy(self, realization_weight):
    strategy = np.maximum(self.regret_sum, 0)
    normalizing_sum = np.sum(strategy)

    if normalizing_sum > 0:
      strategy /= normalizing_sum
    else:
      strategy += np.repeat(1/NUM_ACTIONS, NUM_ACTIONS)

    self.strategy_sum += realization_weight * strategy
    return strategy

  def get_average_strategy(self):
    total = np.sum(self.strategy_sum)
    if total > 0:
      return self.strategy_sum / total
    else:
      return np.repeat(1/NUM_ACTIONS, NUM_ACTIONS)

In [16]:
class KuhnPoker:
  def __init__(self):
    self.cards = np.array([0, 1, 2])
    self.nodes = {}

  def train(self, n=1000):
    for i in range(n):
      shuffle(self.cards)
      self.cfr("", 1, 1)

  def cfr(self, history, p1_reach_prob, p2_reach_prob):
    player = len(history) % 2
    if self.is_terminal(history):
      return self.calculate_payoff(history)

    info_set = f"{self.cards[player]}{history}"
    node = self.get_node(info_set)
    strategy = node.get_strategy(p1_reach_prob if player == 0 else p2_reach_prob)
    action_probs = strategy.copy()
    action_probs *= p1_reach_prob if player == 0 else p2_reach_prob

    node_util = 0
    action_utils = np.zeros(2)
    for action in range(2):
        next_history = f"{history}{action_map[action]}"
        if player == 0:
            action_utils[action] = -self.cfr(next_history, action_probs[action], p2_reach_prob)
        else:
            action_utils[action] = -self.cfr(next_history, p1_reach_prob, action_probs[action])
        node_util += strategy[action] * action_utils[action]

    reach_prob_opponent = p2_reach_prob if player == 0 else p1_reach_prob
    node.regret_sum += (action_utils - node_util) * reach_prob_opponent

    return node_util

  @staticmethod
  def is_terminal(history):
    return len(history) > 1 and history[-2:] in ['pp', 'bb', 'bp']

  def calculate_payoff(self, history):
    player_card = self.cards[len(history) % 2]
    opponent_card = self.cards[1 - len(history) % 2]

    if history[-1] == 'p':
        return -1 if history[-2] == 'p' and player_card < opponent_card else 1
    elif history[-1] == 'b':
        return 2 if player_card > opponent_card else -2

  def get_node(self, info_set):
    if not self.nodes.get(info_set):
      self.nodes[info_set] = Node()
    self.nodes[info_set].visits += 1
    return self.nodes[info_set]

  def print_strategy(self):
    for info_set, node in self.nodes.items():
      print(f"Info Set {info_set}: {np.round(node.get_average_strategy(), decimals=2)}, {node.visits}")


In [18]:
game = KuhnPoker()
game.train(n=1000000)
game.print_strategy()

Info Set 2: [0.18 0.82], 333462
Info Set 0p: [0.67 0.33], 333380
Info Set 2pb: [0. 1.], 333462
Info Set 0b: [1. 0.], 333380
Info Set 0: [0.73 0.27], 333277
Info Set 1p: [1. 0.], 333228
Info Set 0pb: [1. 0.], 333277
Info Set 1b: [0.67 0.33], 333228
Info Set 1: [1. 0.], 333261
Info Set 1pb: [0.39 0.61], 333261
Info Set 2p: [0. 1.], 333392
Info Set 2b: [0. 1.], 333392
